# Data preprocessing

In [ ]:
import os
from keras.preprocessing.image import ImageDataGenerator

''' INSERT THE DIRECTORY WHERE YOU UNZIP THE DATA FILE'''

base_dir = r''
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Rescale all images by the rescale factor
rescale_factor = 1./255

# Training data will be augmented to mitigate overfitting
train_datagen = ImageDataGenerator(
                rescale= rescale_factor,
                rotation_range= 40,
                width_shift_range= 0.2,
                height_shift_range= 0.2,
                shear_range= 0.2,
                zoom_range= 0.2,
                horizontal_flip= True)

valid_datagen = ImageDataGenerator(rescale= rescale_factor)
test_datagen = ImageDataGenerator(rescale= rescale_factor)

''' Three generators set up for three different datasets
    1. Training dataset
    2. Validation dataset
    3. Test dataset
'''

train_generator = train_datagen.flow_from_directory(
                        train_dir, 
                        target_size= (150, 150),
                        batch_size= 30,
                        class_mode= 'categorical')

valid_generator = valid_datagen.flow_from_directory(
                        validation_dir, 
                        target_size= (150, 150),
                        batch_size= 10,
                        class_mode= 'categorical')

test_generator = test_datagen.flow_from_directory(
                        test_dir, 
                        target_size= (150, 150),
                        batch_size= 10,
                        class_mode= 'categorical')

# Instantiating a model for face shape classification 

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()

model.add(layers.Conv2D(64, (3,3), activation= 'relu', input_shape= (150,150, 3)))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(128, (3,3), activation= 'relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(512, (3,3), activation= 'relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(512, (3,3), activation= 'relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation= 'relu'))
model.add(layers.Dense(5, activation= 'softmax'))

In [ ]:
model.summary()

# Compiling & Fitting the model

In [ ]:

from keras import optimizers
model.compile(loss= 'categorical_crossentropy',
              optimizer= optimizers.RMSprop(lr=1e-4),
              metrics= ['acc'])

In [ ]:

STEP_SIZE_TRAIN= train_generator.n//train_generator.batch_size
STEP_SIZE_VALID= valid_generator.n//valid_generator.batch_size


history = model.fit_generator(
            train_generator, 
            steps_per_epoch= STEP_SIZE_TRAIN,
            epochs= 60, 
            validation_data= valid_generator,
            validation_steps = STEP_SIZE_VALID)
    

# Plotting the results

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc'] 
val_acc = history.history['val_acc'] 
loss = history.history['loss'] 
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc') 
plt.plot(epochs, val_acc, 'b', label='Validation acc') 
plt.title('Training and validation accuracy') 
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss') 
plt.title('Training and validation loss') 
plt.legend()
plt.show()


# Evaluate the model with the test dataset

In [ ]:
STEP_SIZE_TEST = test_generator.n//test_generator.batch_size

test_loss, test_acc = model.evaluate_generator(test_generator, steps= STEP_SIZE_TEST )
print('Accuracy on the test dataset: ', round(test_acc*100), '%')